In [505]:
import pandas as pd
from bqplot import *
from dateutil.parser import parse
from ipywidgets import interact, interactive, fixed, interact_manual,Layout,Box
import ipywidgets as widgets
from IPython.display import display

In [378]:
df = pd.read_csv('data/sku_comment_20170110_200k_bfd.csv')
df['comment_create_tm'] = pd.to_datetime(df.comment_create_tm)
df['shelves_dt'] = pd.to_datetime(df.shelves_dt)

In [379]:
period = widgets.Dropdown(options=('S','T','H','D','W','M','Q'),value='D')
how = widgets.Dropdown(options=('sum','mean'),value='sum')
ts_shelve = df.groupby('shelves_dt').count()['comment_id'].reset_index()
ts_shelve.columns = ['date','count']
ts_shelve.index = ts_shelve.date
ts_score = df.groupby('shelves_dt').sum()['score'].reset_index()
ts_score.columns = ['date','sum']
ts_score.index = ts_score.date

In [671]:
class LineChart(object):
    def __init__(self,**args):
        self.df = args.get('df')
        self.x = args.get('x')
        self.y = args.get('y')
        self.sc_x = DateScale()
        self.sc_y = LinearScale()
        self.v_period = 'Q'
        self.v_how = 'sum'
        self._title = ''
        self.charts = []
        self.df_agg = getattr(self.df.resample(self.v_period),self.v_how)().reset_index()
        line = Lines(x=self.df_agg[self.x], y=self.df_agg[self.y], scales={'x':self.sc_x,'y':self.sc_y})
        self.charts.append(line)
        self.ax_x = Axis(scale=dt_x,tick_format='%Y/%m/%d', grid_lines='solid')
        self.ax_y = Axis(scale=sc_y, orientation='vertical')
        self.fig = Figure(marks=self.charts, axes=[self.ax_x, self.ax_y],title='test')

    
    def show(self):
        self.box_layout = Layout(display='flex',flex_flow='row nowrap',border='solid',width='100%')
        self.children = [self.fig]
        self.box = Box(children= self.children, layout=self.box_layout)
        display(self.box)
        
    def setting(self):
        self.period = widgets.Dropdown(options=('S','T','H','D','W','M','Q'),value='Q')
        self.how = widgets.Dropdown(options=('sum','mean','median','max','min'),value='sum')
        self._title = widgets.Text('please set chart title')
        self.period.observe(self.getPeriod, 'value')
        self.how.observe(self.getHow, 'value')
        
        display(self.period,self.how,self._title)
        self._title.observe(self.getTitle,'value')
 
    def getPeriod(self,change):
        self.v_period = change['new']
        self.df_agg = getattr(self.df.resample(self.v_period),self.v_how)()
        self.charts[0].x = self.df_agg.index.values
        self.charts[0].y = self.df_agg[self.y].values

    def getHow(self,change):
        self.v_how = change['new']

        self.df_agg = getattr(self.df.resample(self.v_period),self.v_how)()
        self.charts[0].y = self.df_agg[self.y].values
    
    def addLayer(self,**args):
        df = args.get('df')
        x = args.get('x')
        y = args.get('y')
        df = getattr(df.resample(self.v_period),self.v_how)().reset_index()
        line = Lines(x=df[x], y=df[y], scales={'x':self.sc_x,'y':self.sc_y})
        self.charts.append(line)
        #self.chart = Figure(marks=[self.line,self.line1], axes=[self.ax_x, self.ax_y])
        #display(self.chart)
        self.fig.marks = self.charts
    
    def getTitle(self,change):
        self.fig.title = change['new']

    def removeLayer(self):
        self.charts.pop()
        self.fig.marks = self.charts
    
    @property
    def title(self):
        return self.fig.title

    @title.setter
    def title(self, name):
        self.fig.title = name

In [735]:

line1 = LineChart(df=ts_shelve,x='date',y='count')
line1.show()
line1.setting()

Box(children=(Figure(axes=[Axis(scale=DateScale(), tick_format=u'%Y/%m/%d'), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'right': 60, 'bottom': 60, 'left': 60}, layout=Layout(min_width=u'125px'), marks=[Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'y': LinearScale(), 'x': DateScale()}, scales_metadata={'y': {'orientation': 'vertical', 'dimension': 'y'}, 'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array(['2011-12-31T00:00:00.000000000', '2012-03-31T00:00:00.000000000',
       '2012-06-30T00:00:00.000000000', '2012-09-30T00:00:00.000000000',
       '2012-12-31T00:00:00.000000000', '2013-03-31T00:00:00.000000000',
       '2013-06-30T00:00:00.000000000', '2013-09-30T00:00:00.000000000',
       '2013-12-31T00:00:00.000000000', '2014-03-31T00:00:00.000000000',
       '2014-06-30T00:00:00.000000000', '2014-09-30T00:00:00.000000000',
       '2014-12-31T00:00:00.000000000', '2015-03-31T00:00:00.000000000',
       '2015-06-30T00:00:00.000000000', '2015-09-30T00:00:00.000000000',
       '2015-12-31T00:00:00.000000000', '2016-03-31T00:00:00.000000000',
       '2016-06-30T00:00:00.000000000', '2016-09-30T00:00:00.000000000',
       '2016-12-31T00:00:00.000000000', '2017-03-31T00:00:00.000000000'], dtype='datetime64[ns]'), y=array([  3.38000000e+00,              nan,   3.38000000e+00,
                    nan,   8.45000000e-01,              nan,
         3.29550000e+01,   1.85900000e+01,   3.29550000e+01,
         1.26750000e+02,   9.97100000e+01,   1.93505000e+02,
         4.98550000e+01,   2.58570000e+02,   7.67260000e+02,
         8.36550000e+02,   2.73695500e+03,   2.44036000e+03,
         8.42296000e+03,   1.42052950e+04,   1.17539500e+04,
         1.43650000e+01]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title=u'test'),), layout=Layout(border=u'solid', display=u'flex', flex_flow=u'row nowrap', width=u'100%'))

Dropdown(index=6, options=('S', 'T', 'H', 'D', 'W', 'M', 'Q'), value='Q')

Dropdown(options=('sum', 'mean', 'median', 'max', 'min'), value='sum')

Text(value=u'please set chart title')

In [742]:
l = line1.charts[0]
l.visible = True
l.labels = ['apple']
l.apply_clip = False
l.selected =[0]
l.display_legend = True

In [655]:
d = line1.children.pop()
line1.box.children = line1.children

In [725]:
x = line1.ax_x
x.visible = True
x.grid_color = 'gray'
x.grid_lines = 'dashed'
x.side = 'bottom'
x.label = 'date'
x.num_ticks = 5
x.label_location = 'end'
x.label_color = 'blue'
x.label_offset = '3ex'

In [649]:
line1.children.append(d)
line1.box.children = line1.box.children = line1.children

In [736]:
line1.fig.padding_x = 0.1
line1.fig.padding_y = 0.1
#line1.fig.min_aspect_ratio = 3
#line1.fig.max_aspect_ratio =3.1

In [694]:
line1.fig.fig_margin =  dict(top=30, bottom=60, left=60, right=60)

In [474]:
l.fill = 'bottom'
l.fill_colors =['blue']
l.fill_opacities =[0.5]
l.stroke_width = 3
l.colors = ['green']
l.opacities = [1]
l.labels_visibility = 'label'
l.labels = ['a']
l.close_path = False
l.marker = 'cross'
l.marker_size = 10


In [482]:
l.name

'Lines'

In [385]:
ts1['count'] = ts1['count'] *0.5
line1.addLayer(df=ts1,x='date',y='count')

In [386]:
line1.removeLayer()

In [420]:
line1.fig.save_png()

In [483]:
line1.fig.padding_x = 0.05

In [484]:
line1.fig.padding_y = 0.05